In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization , Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("omkargurav/face-mask-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/face-mask-dataset


In [3]:
with_mask_path='/kaggle/input/face-mask-dataset/data/with_mask'
without_mask_path='/kaggle/input/face-mask-dataset/data/without_mask'

In [4]:
with_mask_images = [os.path.join(with_mask_path, i) for i in os.listdir(with_mask_path)]
without_mask_images = [os.path.join(without_mask_path, i) for i in os.listdir(without_mask_path)]


In [5]:
with_mask_labels=[1]*len(with_mask_images)
without_mask_labels=[0]*len(without_mask_images)

In [6]:
images=with_mask_images+without_mask_images
labels=with_mask_labels+without_mask_labels

In [7]:
df=pd.DataFrame({'files':images,'class':labels}).astype(str)

In [8]:
df.head()

,files,class
0,/kaggle/input/face-mask-dataset/data/with_mask...,1
1,/kaggle/input/face-mask-dataset/data/with_mask...,1
2,/kaggle/input/face-mask-dataset/data/with_mask...,1
3,/kaggle/input/face-mask-dataset/data/with_mask...,1
4,/kaggle/input/face-mask-dataset/data/with_mask...,1


In [9]:
df_train,df_valid =train_test_split(df,test_size=0.2,shuffle=True,stratify=df['class'])

In [10]:
datagen = ImageDataGenerator(rescale=1./255)

In [11]:
train_gen=datagen.flow_from_dataframe(
    dataframe=df_train,
    x_col='files',
    y_col='class',
    target_size=(224,224),
    class_mode='binary',
    batch_size=32
    )

Found 6042 validated image filenames belonging to 2 classes.


In [12]:
test_gen = datagen.flow_from_dataframe(
    dataframe=df_valid,
    x_col='files',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 1511 validated image filenames belonging to 2 classes.


In [13]:
vgg16_model=VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
for layer in vgg16_model.layers:
    layer.trainable = False

In [15]:
model = Sequential([
    vgg16_model,
    Flatten(),
    Dense(514),
    BatchNormalization(),
    Activation('relu'),
    Dense(214),
    BatchNormalization(),
    Activation('relu'),
    Dense(128),
    BatchNormalization(),
    Activation('relu'),
    Dense(64),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Output layer
])

In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 514)            │    12,895,746 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 514)            │         2,056 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 514)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 214)            │       110,210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 214)            │           856 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 214)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        27,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,760,165 (105.90 MB)

 Trainable params: 13,043,637 (49.76 MB)

 Non-trainable params: 14,716,528 (56.14 MB)

In [17]:
model.compile(optimizer=Adam(learning_rate=1e-4),loss='binary_crossentropy',metrics=['accuracy'])

In [18]:
model.fit(train_gen,validation_data=test_gen,epochs=3)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3
 17/189 ━━━━━━━━━━━━━━━━━━━━ 31s 185ms/step - accuracy: 0.6431 - loss: 0.7056

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


189/189 ━━━━━━━━━━━━━━━━━━━━ 78s 325ms/step - accuracy: 0.8490 - loss: 0.3537 - val_accuracy: 0.9768 - val_loss: 0.1151
Epoch 2/3
189/189 ━━━━━━━━━━━━━━━━━━━━ 44s 233ms/step - accuracy: 0.9829 - loss: 0.0894 - val_accuracy: 0.9788 - val_loss: 0.0701
Epoch 3/3
189/189 ━━━━━━━━━━━━━━━━━━━━ 42s 220ms/step - accuracy: 0.9942 - loss: 0.0481 - val_accuracy: 0.9788 - val_loss: 0.0616


In [19]:
model.save('Face_mask_classifier_model.h5')
